# Notebook configuration

In [ ]:
save_plots    = True
plots_format  = "pdf"
outputfolder  = "plots/"

# It's mandatory to have path sctructure as: /what/ever/this/path/is/RUN_NUMBER/ and just the files you want to
# use in this folder. If it's not the case, you should implement proper changes
input_path    = "KDSTPATH"

Zrange        =     0, 600
XYrange       =  -198, 198
Rrange        =     0, max(XYrange)
Erange_raw    =   3e3, 15e3

E0guess       = 13e3
Ltguess       = 1000
R_fiducial    = 50
Z_fiducial    = 10 , 570
Zrange_DV     = 500, 600
nslices       = 20
nStimeprofile = 1800



In [ ]:
import os
import glob
#import shutil
#import functools
import time
print("Running on ", time.asctime())

import numpy             as np
import tables            as tb
import matplotlib.pyplot as plt
from   matplotlib.colors import LogNorm
import matplotlib.dates  as md
import datetime

import invisible_cities.core.core_functions as coref
import invisible_cities.core.fit_functions  as fitf

import invisible_cities.reco.corrections    as corrf
import invisible_cities.io  .dst_io         as dstio

from invisible_cities.icaro.hst_functions import hist
from invisible_cities.icaro.hst_functions import hist2d
from invisible_cities.icaro.hst_functions import labels
from invisible_cities.icaro.hst_functions import shift_to_bin_centers
from invisible_cities.core.stat_functions import poisson_sigma



from invisible_cities.icaro.hst_functions import plot_writer

pi   = np.pi
save = plot_writer(outputfolder, plots_format) if save_plots else lambda *x: None

### General ploting options

In [ ]:
%matplotlib inline

np.warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"]          = 8, 6
plt.rcParams["font.size"]               = 15
plt.rcParams["figure.max_open_warning"] = 100

profOpt = "--k"
fitOpt  = "r"
if not os.path.exists(outputfolder):
    os.mkdir(outputfolder)

# Read data

In [ ]:
full = dstio.load_dsts(glob.glob(input_path + '*h5'), "DST", "Events")
full = full[full.R<max(Rrange)] # soft radial cut to reduce badly reconstructed events
print(len(full))
full.head()

In [ ]:
fid = full
fid = fid[fid.R < R_fiducial] # Fiducialize in order to reduce geometrical effects
print("Ratio:", len(fid)/len(full))
fid.head()

# Lifetime computation

### General plots

In [ ]:
hist(fid.S2e, 50, range=Erange_raw)
labels("S2 energy (pes)", "Entries", "Fidualized energy spectrum")
save("Raw_energy_fiducial")


hist2d(fid.Z, fid.S2e, (100, 50), range=(Zrange, Erange_raw))
labels("Drift time ($\mu$s)", "S2 energy (pes)", "Fiducialized energy spectrum vs Z")
save("Raw_energy_vsZ_fiducial")


## Computation

In [ ]:
# Data selection

data = fid

Zbins = np.linspace(*Z_fiducial, nslices)
Zcorr = corrf.LifetimeCorrection(-Ltguess, E0guess)


Z_drift_data = []
for i in range(len(Zbins)-1):
    zpos    = np.mean([Zbins[i], Zbins[i+1]])
    data_   = data[coref.in_range(data.Z.values, Zbins[i], Zbins[i+1])]
    E_zcorr = data_.S2e.values 
    
    y, x    = np.histogram(E_zcorr, 50, range=Erange_raw)
    x       = shift_to_bin_centers(x)
    seed = 1e4,  E0guess*Zcorr(zpos).value, 200 
    f    = fitf.fit(fitf.gauss, x, y, seed, fit_range=Erange_raw)
    Z_drift_data.append([f.values[1],f.errors[1]])


In [ ]:
E_z = data.S2e 
hist2d(data.Z, E_z, (100, 50), range=(Zrange, Erange_raw))

x, y, u_y = map(np.array, [Zbins, *zip(*Z_drift_data)])
x = shift_to_bin_centers(x)

plt.errorbar(x, y, yerr=u_y, fmt='b', linewidth=2)

seed = E0guess, -Ltguess
f    = fitf.fit(fitf.expo, x, y, seed, fit_range=Z_fiducial, sigma=u_y)

E0mean, Ltmean = f.values 

plt.plot(x, f.fn(x), c='r', linewidth=2)
print("Energy at z=0 = {:.1f} +- {:.1f}".format( f.values[0], f.errors[0]))
print("Lifetime      = {:.3f} +- {:.3f}".format(-f.values[1], f.errors[1]))
print("Chi2          = {:.2f}          ".format(f.chi2))

labels("Drift time ($\mu$s)", "S2 energy (pes)", "Fiducialized energy spectrum vs Z")
save("S2energyvsZ_fiducial")

### time profile

In [ ]:
data = fid
data = data.sort_values('event')

tstart = data.time.values[ 0]
tfinal = data.time.values[-1]

nStimeprofile

ntimebins = int( np.floor( ( tfinal - tstart) / nStimeprofile) )
ntimebins = np.max([ntimebins, 1])

tbins = np.linspace( tstart, tfinal, ntimebins+1)
time, lt, elt = [], [], []

for t in range(ntimebins):
    data_time = data[coref.in_range(data.time, tbins[t], tbins[t+1])]
    Zbins = np.linspace(*Z_fiducial, nslices)
    Zcorr = corrf.LifetimeCorrection(Ltmean, E0mean)
    Z_drift_data = []
    for i in range(len(Zbins)-1):
        zpos    = np.mean([Zbins[i], Zbins[i+1]])
        data_   = data_time[coref.in_range(data_time.Z.values, Zbins[i], Zbins[i+1])]
        E_zcorr = data_.S2e.values 

        y, x    = np.histogram(E_zcorr, 50, range=Erange_raw)
        x       = shift_to_bin_centers(x)
        seed = 1e4,  E0guess*Zcorr(zpos).value, 100 
        f    = fitf.fit(fitf.gauss, x, y, seed, fit_range=Erange_raw)
        Z_drift_data.append([f.values[1],f.errors[1]])
    x, y, u_y = map(np.array, [Zbins, *zip(*Z_drift_data)])
    x = shift_to_bin_centers(x)
    
    #plt.figure()
    #hist2d(data_time.Z, data_time.S2e, (100, 50), range=(Zrange, Erange_raw))
    #plt.errorbar(x,y)
    
    seed = E0mean, Ltmean
    f    = fitf.fit(fitf.expo, x, y, seed, fit_range=Z_fiducial, sigma=u_y)
    time.append(np.mean([tbins[t],tbins[t+1]]))
    lt  .append(-f.values[1])
    elt .append(-f.errors[1])


time_ = list(map(datetime.datetime.fromtimestamp, time))

plt.figure()
ax=plt.gca()
xfmt = md.DateFormatter('%d-%m %H:%M')
ax.xaxis.set_major_formatter(xfmt)    

plt.errorbar(time_, lt, elt)
plt.xlabel('date')
plt.ylabel('Lifetime (mus)')
plt.xticks( rotation=25 )
save("LifetimevsDate")


# Drift velocity

In [ ]:
data = fid
y, x, _ = hist(data.Z, 50, Zrange_DV)
 
sigmoid  = lambda x, A, B, C, D: A/(1 + np.exp(-C*(x-B))) + D
seed     = np.max(y), np.mean(Zrange_DV), np.diff(Zrange_DV)[0]/100, np.min(y)
f        = fitf.fit(sigmoid, x, y, seed, sigma=poisson_sigma(y), fit_range=Zrange_DV)

dv   = 530.2/f.values[1]
u_dv =  dv/f.values[1] * f.errors[1]

plt.plot(x, f.fn(x), fitOpt)

labels("Drift time ($\mu$s)", "Entries", "Z distribution")
print("Max drift length = {:.3f} +- {:.3f}".format(f.values[1], f.errors[1]))
print("Drift velocity   = {:.5f} +- {:.5f}".format(dv, u_dv))
save("Drift_velocity")